In [ ]:
from OpenGL.GL import *
from OpenGL.GLUT import *
from OpenGL.GLU import *
import random
import math
import sys
W_Width = 500
W_Height = 500
spaceship_x = W_Width / 2
spaceship_y = 20
spaceship_speed = 8
projectiles = []
projectile_speed = 4
falling_circles = []
circle_radius = 10
circle_speed = 0.1
score = 0
consecutive_misses = 0
missed_shots = 0
game_over = False
paused = False
game_started = False
game_over_reason = ""
def draw_circle_midpoint(cx, cy, radius, color):
    glColor3f(color[0], color[1], color[2])
    glBegin(GL_POINTS)
    x, y = 0, radius
    p = 1 - radius
    def draw_symmetric_points(xc, yc, x, y):
        glVertex2f(xc + x, yc + y)
        glVertex2f(xc - x, yc + y)
        glVertex2f(xc + x, yc - y)
        glVertex2f(xc - x, yc - y)
        glVertex2f(xc + y, yc + x)
        glVertex2f(xc - y, yc + x)
        glVertex2f(xc + y, yc - x)
        glVertex2f(xc - y, yc - x)
    draw_symmetric_points(cx, cy, x, y)
    while(x < y):
        x += 1
        if(p < 0):
            p += 2 * x + 1
        else:
            y -= 1
            p += 2 * (x - y) + 1
        draw_symmetric_points(cx, cy, x, y)
    glEnd()
def draw_line_midpoint(x1, y1, x2, y2, color):
    glColor3f(color[0], color[1], color[2])
    glBegin(GL_POINTS)
    dx = abs(x2 - x1)
    dy = abs(y2 - y1)
    x, y = x1, y1
    if x2 > x1:
        sx = 1
    else:
        sx = -1

    if y2 > y1:
        sy = 1
    else:
        sy = -1

    if(dx > dy):
        err = dx / 2.0
        while(x != x2):
            glVertex2f(x, y)
            err -= dy
            if(err < 0):
                y += sy
                err += dx
            x += sx
    else:
        err = dy / 2.0
        while(y != y2):
            glVertex2f(x, y)
            err -= dx
            if(err < 0):
                x += sx
                err += dy
            y += sy
    glVertex2f(x, y)
    glEnd()
def init():
    glClearColor(0, 0, 0, 0)
    glMatrixMode(GL_PROJECTION)
    glLoadIdentity()
    glOrtho(0.0, W_Width, 0.0, W_Height, 0.0, 1.0)
    glMatrixMode(GL_MODELVIEW)
def draw_spaceship():
    spaceship_width = 20
    spaceship_height = 15
    square_size = 20
    x1 = spaceship_x - square_size / 2
    y1 = spaceship_y
    x2 = spaceship_x + square_size / 2
    y2 = spaceship_y + square_size
    x3 = spaceship_x
    y3 = y2 + spaceship_height
    draw_line_midpoint(int(x1), int(y1), int(x2), int(y1), (0.0, 0.5, 1.0))
    draw_line_midpoint(int(x2), int(y1), int(x2), int(y2), (0.0, 0.5, 1.0))
    draw_line_midpoint(int(x2), int(y2), int(x1), int(y2), (0.0, 0.5, 1.0))
    draw_line_midpoint(int(x1), int(y2), int(x1), int(y1), (0.0, 0.5, 1.0))
    draw_line_midpoint(int(x1), int(y2), int(x3), int(y3), (1.0, 1.0, 0.0))
    draw_line_midpoint(int(x3), int(y3), int(x2), int(y2), (1.0, 1.0, 0.0))
    draw_line_midpoint(int(x2), int(y2), int(x1), int(y2), (1.0, 1.0, 0.0))
def draw_falling_circles():
    for cx, cy in falling_circles:
        draw_circle_midpoint(int(cx), int(cy), circle_radius, (1.0, 1.0, 0.0))
def draw_projectile():
    for px, py in projectiles:
        draw_circle_midpoint(int(px), int(py), 2, (1.0, 0.0, 0.0))
def draw_score():
    glColor3f(1.0, 1.0, 1.0)
    glBegin(GL_QUADS)
    glVertex2f(W_Width - 90, W_Height - 30)
    glVertex2f(W_Width - 10, W_Height - 30)
    glVertex2f(W_Width - 10, W_Height - 60)
    glVertex2f(W_Width - 90, W_Height - 60)
    glEnd()
    glColor3f(0.0, 0.0, 0.0)
    render_text(W_Width - 80, W_Height - 50, f"Score: {score}")
def render_text(x, y, text):
    glRasterPos2f(x, y)
    for i in text:
        glutBitmapCharacter(GLUT_BITMAP_8_BY_13, ord(i))
def update_game():
    global projectiles
    global falling_circles
    global score
    global consecutive_misses
    global game_over
    global missed_shots
    if game_over:
        return
    if not paused:
        for projectile in projectiles[:]:
            projectile[1] += projectile_speed
            if(projectile[1] > W_Height):
                projectiles.remove(projectile)
                missed_shots += 1
        for projectile in projectiles[:]:
            px, py = projectile
            for i in falling_circles[:]:
                cx, cy = i
                if((px - cx) ** 2 + (py - cy) ** 2 <= circle_radius ** 2):
                    falling_circles.remove(i)
                    projectiles.remove(projectile)
                    score += 1
                    consecutive_misses = 0
                    print(f"Score: {score}")
                    break
        if(random.random() < 0.02):
            x_pos = random.randint(circle_radius, W_Width - circle_radius)
            falling_circles.append((x_pos, W_Height))
        new_falling_circles = []
        for i in falling_circles[:]:
            cx, cy = i
            if(cy - circle_radius <= spaceship_y + 15 and spaceship_x - 10 < cx < spaceship_x + 10):
                game_over = True
                global game_over_reason
                game_over_reason = "A circle hit the spaceship!"
                print("Game Over! A circle hit the spaceship!")
                break
            if(cy - circle_radius <= 0):
                consecutive_misses += 1
            else:
                new_falling_circles.append((cx, cy - circle_speed))
        falling_circles = new_falling_circles
        if(consecutive_misses >= 3):
            game_over = True
            game_over_reason = "You missed 3 circles!"
            print("Game Over! You missed 3 circles!")
            print(f"Final Score: {score}")
        if(missed_shots >= 3):
            game_over = True
            game_over_reason = "You misfired 3 times!"
            print("Game Over! You misfired 3 times!")
            print(f"Final Score: {score}")
def keyboardListener(key, x, y):
    global spaceship_x, projectiles, spaceship_speed, game_started, game_over, score, game_over_reason
    if key == b'\r':
        if game_over:
            reset_game()
        else:
            game_started = True
        glutPostRedisplay()
    if(key == b'\x1b'):
        glutLeaveMainLoop()
    if game_started and not game_over:
        if(key == b'a'):
            spaceship_x -= spaceship_speed
            if(spaceship_x < 0):
                spaceship_x = 0
        if(key == b'd'):
            spaceship_x += spaceship_speed
            if(spaceship_x > W_Width):
                spaceship_x = W_Width
        if(key == b' ' and not game_over):
            projectiles.append([spaceship_x, spaceship_y + 15])
def mouseClick(button, state, x, y):
    global paused
    if(10 < x < 70 and W_Height - 30 < y < W_Height - 10):
        paused = not paused
        glutPostRedisplay()
def display():
    glClear(GL_COLOR_BUFFER_BIT | GL_DEPTH_BUFFER_BIT)
    glLoadIdentity()
    if not game_started:
        draw_start_screen()
    elif game_over:
        draw_game_over_screen()
    else:
        draw_spaceship()
        draw_falling_circles()
        draw_projectile()
        update_game()
        draw_score()
        draw_pause_button()
    glutSwapBuffers()
def draw_pause_button():
    glColor3f(1.0, 1.0, 1.0)
    if paused:
        render_text(10, W_Height - 30, "Resume")
    else:
        render_text(10, W_Height - 30, "Pause")
def draw_start_screen():
    glColor3f(1.0, 1.1, 1.0)
    render_text(W_Width / 4, W_Height / 2 + 50, "Space Shooter Made by Ahmed Rafiun Nur")
    render_text(W_Width / 4, W_Height / 2 + 20, "Space Shooting Game")
    render_text(W_Width / 4, W_Height / 2 - 20, "Press Enter to Play")
    render_text(W_Width / 4, W_Height / 2 - 50, "Press Escape to Exit")
def draw_game_over_screen():
    glColor3f(1.0, 1.0, 1.0)  # White color for text
    render_text(W_Width / 4, W_Height / 2 + 40, "Game Over!")
    render_text(W_Width / 4, W_Height / 2 + 20, f"Score: {score}")
    render_text(W_Width / 4, W_Height / 2, f"Reason: {game_over_reason}")
    render_text(W_Width / 4, W_Height / 2 - 20, "Press Enter to Play Again")
    render_text(W_Width / 4, W_Height / 2 - 40, "Press Escape to Exit")
def reset_game():
    global spaceship_x, spaceship_y, projectiles, falling_circles, score, consecutive_misses, missed_shots, game_over, game_over_reason
    spaceship_x = W_Width / 2
    spaceship_y = 20
    projectiles = []
    falling_circles = []
    score = 0
    consecutive_misses = 0
    missed_shots = 0
    game_over = False
    game_over_reason = ""
def init_game():
    glutInit(sys.argv)
    glutInitDisplayMode(GLUT_RGBA | GLUT_DOUBLE)
    glutInitWindowSize(W_Width, W_Height)
    glutCreateWindow(b"Space Shooting Game")
    init()
    glutDisplayFunc(display)
    glutKeyboardFunc(keyboardListener)
    glutMouseFunc(mouseClick)
    glutIdleFunc(display)
    glutMainLoop()
init_game()

